In [3]:
import os
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from torchvision.ops import FeaturePyramidNetwork


In [4]:
model = EfficientNet.from_pretrained('efficientnet-b0')
model

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [5]:

class EfficientNetFPN(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetFPN, self).__init__()
        
        # 加载预训练的EfficientNet模型
        self.backbone = EfficientNet.from_pretrained('efficientnet-b0')
        
        # 使用Feature Pyramid Network
        self.fpn = FeaturePyramidNetwork(in_channels_list=[40, 80, 112, 320],
                                          out_channels=256)
        
        # 分类层
        self.fc1 = nn.Linear(256, num_classes)
        self.fc2 = nn.Linear(256, num_classes)
        self.fc3 = nn.Linear(256, num_classes)
        self.fc4 = nn.Linear(256, num_classes)

    def forward(self, x):
        # EfficientNet部分
        x = self.backbone.extract_features(x)  # 获取每一层的特征

        # 把特征按照不同的维度切分
        x1, x2, x3, x4 = torch.split(x, [40, 80, 112, 320], dim=1)
        
        # FPN部分
        features = self.fpn({"0": x1, "1": x2, "2": x3, "3": x4})
        
        out1 = F.adaptive_avg_pool2d(features["0"], (1, 1))
        out1 = torch.flatten(out1, 1)
        out1 = self.fc1(out1)
        
        out2 = F.adaptive_avg_pool2d(features["1"], (1, 1))
        out2 = torch.flatten(out2, 1)
        out2 = self.fc2(out2)
        
        out3 = F.adaptive_avg_pool2d(features["2"], (1, 1))
        out3 = torch.flatten(out3, 1)
        out3 = self.fc3(out3)
        
        out4 = F.adaptive_avg_pool2d(features["3"], (1, 1))
        out4 = torch.flatten(out4, 1)
        out4 = self.fc4(out4)
        
        # 你可以选择如何组合这四个输出，这里我选择的是平均
        out = (out1 + out2 + out3 + out4) / 4.0
        
        return out

# import os
# import torch
# from torch import nn
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader
# from efficientnet_pytorch import EfficientNet
# from torchvision.ops import FeaturePyramidNetwork

# class EfficientNetFPN(nn.Module):
#     def __init__(self, num_classes):
#         super(EfficientNetFPN, self).__init__()
        
#         # 加载预训练的EfficientNet模型
#         self.backbone = EfficientNet.from_pretrained('efficientnet-b0')
        
#         # 获取EfficientNet中的不同层的输出大小
#         self.stage1 = nn.Sequential(self.backbone._conv_stem, self.backbone._bn0, self.backbone._blocks[:3])
#         self.stage2 = self.backbone._blocks[3:5]
#         self.stage3 = self.backbone._blocks[5:10]
#         self.stage4 = self.backbone._blocks[10:]
        
#         # 使用Feature Pyramid Network
#         self.fpn = FeaturePyramidNetwork(in_channels_list=[40, 80, 112, 320],
#                                           out_channels=256)
        
#         # 分类层
#         self.fc1 = nn.Linear(256, num_classes)
#         self.fc2 = nn.Linear(256, num_classes)
#         self.fc3 = nn.Linear(256, num_classes)
#         self.fc4 = nn.Linear(256, num_classes)

#     def forward(self, x):
#         # EfficientNet部分
#         x1 = self.stage1(x)
#         x2 = self.stage2(x1)
#         x3 = self.stage3(x2)
#         x4 = self.stage4(x3)
        
#         # FPN部分
#         features = self.fpn({"0": x1, "1": x2, "2": x3, "3": x4})
        
#         out1 = F.adaptive_avg_pool2d(features["0"], (1, 1))
#         out1 = torch.flatten(out1, 1)
#         out1 = self.fc1(out1)
        
#         out2 = F.adaptive_avg_pool2d(features["1"], (1, 1))
#         out2 = torch.flatten(out2, 1)
#         out2 = self.fc2(out2)
        
#         out3 = F.adaptive_avg_pool2d(features["2"], (1, 1))
#         out3 = torch.flatten(out3, 1)
#         out3 = self.fc3(out3)
        
#         out4 = F.adaptive_avg_pool2d(features["3"], (1, 1))
#         out4 = torch.flatten(out4, 1)
#         out4 = self.fc4(out4)
        
#         # 你可以选择如何组合这四个输出，这里我选择的是平均
#         out = (out1 + out2 + out3 + out4) / 4.0
        
#         return out
    

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 加载数据集
data_dir = './MO_106/'  # 替换为你的图片文件夹路径
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 初始化模型和优化器
model = EfficientNetFPN(num_classes=10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 训练模型
for epoch in range(10):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# 测试模型
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Test Accuracy: {}%'.format(100 * correct / total))


Loaded pretrained weights for efficientnet-b0


RuntimeError: split_with_sizes expects split_sizes to sum exactly to 1280 (input tensor's size at dimension 1), but got split_sizes=[40, 80, 112, 320]